# 3. Topic labelling

## Setup
As always, we will begin by loading a set of constants and initializing the logging system. Since we will be using Bokeh in this notebook, we will configure it to output the results in the Jupyter notebook:

In [1]:
%run __init__.py

In [2]:
from bokeh.io import output_notebook

output_notebook()

/home/alejandro/.envs/edma/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


Loading BokehJS ...

In [3]:
import pandas as pd

GIT_FILE_PATH = os.path.join(NOTEBOOK_1_RESULTS_DIR, 'git_dataframe.pkl')

git_df = pd.read_pickle(GIT_FILE_PATH)
git_repositories = git_df['full_text_cleaned'].values

## Entity linking

### Using the entity linking class
An entity linking class has been defined in the _entity_linking.py_ module of the _src_ directory. This class will link the given words to their Wikidata entity by using the [wbsearchentities](https://www.wikidata.org/w/api.php?action=help&modules=wbsearchentities) modules from the MediaWiki API:

In [4]:
from herc_common.entity_linking import WikidataEntityLinker

entity_linker = WikidataEntityLinker()
res = entity_linker.link_entity('python')
res

('python', 'http://www.wikidata.org/entity/Q28865')

### Linking each topic's term to Wikidata
In the following cells we are going to load the lda model trained on the Agriculture dataset, obtain the term distribution of each topic, and link each term to Wikidata. We will start by loading both the LDA pipeline and the document term matrix with the term frequency: 

In [5]:
from herc_common.utils import load_object

lda_agriculture_pipe_filename = "git_nmf_model.pkl"
dtm_tf_filename = "git_dtm_tfidf.pkl"

lda_pipe = load_object(os.path.join(NOTEBOOK_2_RESULTS_DIR, lda_agriculture_pipe_filename))
dtm_tf = load_object(os.path.join(NOTEBOOK_2_RESULTS_DIR, dtm_tf_filename))

In order to obtain the list of terms for each topic, we are going to make use of the _get\_topic\_terms\_by\_relevance_ function to obtain a list of more relevant terms for each topic (see [Sievert & Shirley](https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf) for more information).

In [6]:
from herc_common.utils import get_topic_terms_by_relevance

def link_topic_terms(entity_linker, model, vectorizer,
                     dtm_tf, n_top_words, lambda_=0.6):
    res = []
    if lambda_ < 1.0:
        topic_terms = get_topic_terms_by_relevance(model, vectorizer, dtm_tf,
                                                   n_top_words, lambda_)
    else:
        feature_names = vectorizer.get_feature_names()
        topic_terms = [[feature_names[i] for i in topic.argsort()[:-n_top_words - 1: -1]] 
                       for topic in model.components_]
    return [[entity_linker.link_entity(entity) for entity in topic]
            for topic in topic_terms]


Finally, we can make used of the function defined above to link each term to Wikidata. The output of the following cell will be a 2D array, with the first dimension corresponding to each topic, and the second one consisting on tuples containing the pair ('term', 'wikidata_uri') for every term of the topic:

In [7]:
linked_terms = link_topic_terms(entity_linker, lda_pipe.named_steps['model'],
                                lda_pipe.named_steps['vectorizer'], dtm_tf, 
                                n_top_words=10, lambda_=1)
linked_terms

[[('base', 'http://www.wikidata.org/entity/Q191360'),
  ('Command', 'http://www.wikidata.org/entity/Q1079196'),
  ('array', 'http://www.wikidata.org/entity/Q186152'),
  ('Index', 'http://www.wikidata.org/entity/Q873506'),
  ('create', 'http://www.wikidata.org/entity/Q30218413'),
  ('Operation', 'http://www.wikidata.org/entity/Q1079196'),
  ('core', 'http://www.wikidata.org/entity/Q23595'),
  ('base base', 'http://www.wikidata.org/entity/Q35556177'),
  ('sesame', 'http://www.wikidata.org/entity/Q18028187'),
  ('dialog', 'http://www.wikidata.org/entity/Q131395')],
 [('biosample', 'http://www.wikidata.org/entity/Q4914649'),
  ('BioSample', 'http://www.wikidata.org/entity/Q4914649'),
  ('BioSample RDF', None),
  ('BioSample record', None),
  ('record', 'http://www.wikidata.org/entity/Q49848'),
  ('RDF', 'http://www.wikidata.org/entity/Q54872'),
  ('BioHackathon', 'http://www.wikidata.org/entity/Q27975958'),
  ('EBI biosample', None),
  ('xml', 'http://www.wikidata.org/entity/Q2115'),
  ('d

## Obtaining each topic's graphs
In this phase we are going to explore the neighbourhood of each term linked before, to obtain a graph with their related terms from Wikidata. Each set of terms obtained before will be the seed concepts used to obtain the final graph, and a set of properties from Wikidata will be explored recursively to expand the final graph. 

For more information about the implementation of the graph building process, the class used can be accessed at the _graph.py_ module in the source directory.

In the following cell we will be configuring the graph builder to build a graph with a maxium depth of two from every seed node. Higher depth values might cause the resulting topic labels to be very general, while with a smaller value we have the risk of not obtaining a connection between the seed nodes:

In [8]:
from herc_common.graph import WikidataGraphBuilder

graph_builder = WikidataGraphBuilder(max_hops=2)
topic_graphs = [graph_builder.build_graph(topic) for topic in linked_terms]

Now that we have obtained the neighbourhood graph of each topic, we are going to plot the results using bokeh. Each node will have a different color depending on their depth with respect to the seed nodes, which will be painted in blue. This will allow us to perform an initial exploration of these graphs:

In [9]:
from bokeh.io import show
from bokeh.layouts import gridplot

from herc_common.bokeh_utils import build_graph_plot


plots = [build_graph_plot(g, f"Topic {idx}") 
         for idx, g in enumerate(topic_graphs)]
grid = gridplot(plots, ncols=2)
show(grid)

An optimum result would be to have every seed term connected in the final graph. However, theere will be some subgraphs which are isolated from the main ones. In the following section we will be solving this issue.

## Getting the main connected subgraph
As we have described before, some of the topic graphs that we have obtained are not fully connected. Small subgraphs which are isolated from the main subgraph will be considered as noise, and removed before the following computations.

In the following cells, we are going to retrieve the largest connected subgraph from each topic's graph, and plot the results to anaylise them:

In [10]:
from herc_common.graph import get_largest_connected_subgraph

connected_topic_subgraphs = [get_largest_connected_subgraph(g) 
                             for g in topic_graphs]

In [11]:
plots = [build_graph_plot(g, f"Largest Connected subgraph for topic {idx}") 
         for idx, g in enumerate(connected_topic_subgraphs)]
grid = gridplot(plots, ncols=2)
show(grid)

In this section we are aiming to see big graphs with the most amount of seed nodes possible. Graphs with few seed nodes from the original term distribution will tend to be less representative of the original topic.

## Obtaining the main component of each topic
Now that we have the final subgraph for each topic, we will be applying several centrality measures to obtain the node that best represents the topic. In the following cell we have defined an auxiliary function that receives a list of algorithm and returns the results of applying them to obtain the best _n_ entities that represent each topic:

In [12]:
import networkx.algorithms as nxa

from herc_common.graph import get_centrality_algorithm_results

def try_centrality_algorithms(topic_subgraphs, algorithms, stop_uris, top_n=4):
    markdown = ""
    for (algorithm, name) in algorithms:
        print(f'Algorithm: {name}')
        results = [get_centrality_algorithm_results(g, algorithm, stop_uris, top_n)
                   for g in topic_subgraphs]
        results_labels = [[(node[0]['label'], node[1]) for node in topic] 
                          for topic in results]
        for idx, result in enumerate(results_labels):
            print(f"Topic {idx}:", result)
            print()
        print()

        
algorithms = [
    (nxa.centrality.information_centrality, "Information centrality"),
    (nxa.centrality.eigenvector_centrality_numpy, "Eigenvector centrality"),
    (nxa.centrality.closeness_centrality, "Closeness centrality"),
    (nxa.centrality.betweenness_centrality, "Betweenness centrality"),
    (nxa.centrality.communicability_betweenness_centrality, "Communicability betweenness centrality")
]

try_centrality_algorithms(connected_topic_subgraphs,
               algorithms,
               ['Q4167836', 'Q11862829'])

Algorithm: Information centrality
Topic 0: [('database', 0.009411764705882354), ('work', 0.008830022075055186), ('index', 0.008438818565400845), ('construction', 0.008179959100204501)]

Topic 1: [('library science', 0.006641208699983401), ('document', 0.006386715631486514), ('text', 0.005729838132072767), ('written work', 0.005684240443370754)]

Topic 2: [('software', 0.007148659626320076), ('application', 0.006825409136740873), ('computer program', 0.006605614772556669), ('biology', 0.006140106056377339)]

Topic 3: [('protein', 0.016107382550335586), ('first-order metaclass', 0.012806830309498397), ('Probable ABC transporter ATP-binding protein MlaF/Mkl', 0.011753183153770816), ('protein family', 0.011483253588516764)]

Topic 4: [('concept', 0.04411764705882353), ('abstract object', 0.03846153846153847), ('variable-order metaclass', 0.037500000000000026), ('mental representation', 0.03260869565217392)]

Topic 5: [('computer science', 0.004768428749835668), ('algorithm', 0.004722786759

Topic 0: [('database', 0.6510243659075945), ('index', 0.3004455834282246), ('collection', 0.2278874842428074), ('work', 0.20345024155795693)]

Topic 1: [('document', 0.46103870226019), ('text', 0.4003996499585014), ('library science', 0.3812737986546991), ('written work', 0.3040224006398725)]

Topic 2: [('biology', 0.5327165168993215), ('software', 0.3455372943243429), ('morphology', 0.2333077365639031), ('computer program', 0.22936723671279993)]

Topic 3: [('protein', 0.704260082621068), ('first-order metaclass', 0.1390066022895114), ('Probable ABC transporter ATP-binding protein MlaF/Mkl', 0.134149835944969), ('microtubule polymerization based protein transport', 0.12410767547874184)]

Topic 4: [('variable-order metaclass', 0.5718113340621223), ('concept', 0.4808719512837688), ('abstract object', 0.45277617728657277), ('mental representation', 0.20683083591952253)]

Topic 5: [('mathematics', 0.615547158520548), ('computer science', 0.2765347935937446), ('algorithm', 0.253050404738871

Topic 0: [('database', 0.35714285714285715), ('work', 0.33613445378151263), ('index', 0.31496062992125984), ('construction', 0.3125)]

Topic 1: [('library science', 0.34375), ('document', 0.3313253012048193), ('life sciences', 0.2894736842105263), ('text', 0.2722772277227723)]

Topic 2: [('software', 0.31901840490797545), ('application', 0.30057803468208094), ('computer program', 0.27956989247311825), ('software category', 0.2708333333333333)]

Topic 3: [('protein', 0.625), ('first-order metaclass', 0.5), ('Probable ABC transporter ATP-binding protein MlaF/Mkl', 0.4326923076923077), ('protein catabolic process, modulating synaptic transmission', 0.3879310344827586)]

Topic 4: [('concept', 0.6), ('abstract object', 0.5), ('variable-order metaclass', 0.4838709677419355), ('mental representation', 0.45454545454545453)]

Topic 5: [('algorithm', 0.2996742671009772), ('mathematics', 0.2948717948717949), ('algorithmics', 0.2920634920634921), ('computer science', 0.2787878787878788)]

Topic 6:

Topic 0: [('database', 0.7153846153846154), ('construction', 0.5692307692307692), ('work', 0.49230769230769234), ('index', 0.4198717948717949)]

Topic 1: [('library science', 0.6565656565656566), ('document', 0.6090909090909091), ('life sciences', 0.5050505050505051), ('BioHackathon series in 2011 and 2012: penetration of ontology and linked data in life science domains', 0.43434343434343436)]

Topic 2: [('software', 0.6225490196078431), ('application', 0.4743589743589744), ('software category', 0.4562594268476622), ('biology', 0.4364630467571645)]

Topic 3: [('protein', 0.897979797979798), ('first-order metaclass', 0.36262626262626263), ('type of statistic', 0.24141414141414141), ('protein family', 0.21616161616161617)]

Topic 4: [('concept', 0.7142857142857143), ('abstract object', 0.37142857142857144), ('mental representation', 0.37142857142857144), ('capability', 0.2571428571428572)]

Topic 5: [('algorithm', 0.4602768866744026), ('algorithmics', 0.3906365890837941), ('mathematics',

Topic 0: [('database', 0.7244654605944721), ('construction', 0.5724281971796762), ('work', 0.508243348161075), ('index', 0.4417445201075427)]

Topic 1: [('library science', 0.6634714603763229), ('document', 0.6288511991749709), ('life sciences', 0.5090541726339515), ('BioHackathon series in 2011 and 2012: penetration of ontology and linked data in life science domains', 0.4396384098187511)]

Topic 2: [('software', 0.6469866506372794), ('application', 0.5049832253970399), ('software category', 0.4604334363217811), ('biology', 0.45582402516060994)]

Topic 3: [('protein', 0.9082605283457162), ('first-order metaclass', 0.40425666678082883), ('type of statistic', 0.2480497165897224), ('protein family', 0.2305518007739436)]

Topic 4: [('concept', 0.7752857657498203), ('abstract object', 0.46782538014128705), ('variable-order metaclass', 0.3961458894054767), ('mental representation', 0.39346531343788416)]

Topic 5: [('algorithm', 0.48458268525315473), ('mathematics', 0.4132272751059948), ('al

## Add labels to LDA model
Finally, we will be saving the best results to our LDA model that has been trained previously. Now, when we load the model again, after a topic has been inferred for a given text we will also be able to return a representative label for the topic, which will be also linked to Wikidata:

In [13]:
from herc_common.topic import Topic

final_results = [get_centrality_algorithm_results(g,
                                                 nxa.centrality.information_centrality,
                                                ['Q4167836', 'Q11862829'], top_n=1)
                 for g in connected_topic_subgraphs]

final_results_topics = [Topic.from_node(topic[0], topic[1], "lda") 
                        for result in final_results for topic in result]
lda_model = lda_pipe.named_steps['model']

In [14]:
from tqdm import tqdm

import en_core_web_md
import string
import numpy as np

en_core_web_md.load()

In [15]:
from herc_common.topic import LabelledTopicModel

labelled_topic_model = LabelledTopicModel(lda_model, final_results_topics)

lda_pipe.steps.pop()
lda_pipe.steps.append(('model', labelled_topic_model))

In [16]:
from herc_common.utils import save_object

save_object(lda_pipe, os.path.join(NOTEBOOK_3_RESULTS_DIR, 'lda_pipe_with_labels.pkl'))

## Obtaining the results for every article in the dataset

In [17]:
import en_core_sci_lg

en_core_sci_lg.load()

In [18]:
results = lda_pipe.transform(git_repositories)

## Saving the results

In [24]:
NEW_COL_NAME = 'topics_from_lda'

git_df[NEW_COL_NAME] = ['\n'.join([f"{topic.label}, {topic.score:.5f}" for topic in result])
                        for result in results]

results_df = git_df[['gh_id', 'name', NEW_COL_NAME]]
results_df.head()

,gh_id,name,topics_from_lda
0,216602979,LIRICAL,"Protein kinase domain, 0.12012\ncodon, 0.00000..."
1,199330464,wikidata_ontomatcher,"Wikidata, 0.03981\ncodon, 0.00000\nUtility lib..."
2,253207181,ro-crate-ruby,"information, 0.08174\ndata science, 0.00000\ng..."
3,212556220,Misc_Training_scripts,"virtuoso, 0.04775\nvirtuoso, 0.02404\ndocument..."
4,155879756,FAIRifier,"database, 0.13379\ntextile, 0.00004\nHyperText..."


In [25]:
OUTPUT_FILE_NAME = "git_df_with_lda_topics.csv"

results_df.to_csv(os.path.join(NOTEBOOK_3_RESULTS_DIR, OUTPUT_FILE_NAME), index=False)

In [34]:
results_df

,gh_id,name,topics_from_lda
0,216602979,LIRICAL,"Protein kinase domain, 0.12012\ncodon, 0.00000..."
1,199330464,wikidata_ontomatcher,"Wikidata, 0.03981\ncodon, 0.00000\nUtility lib..."
2,253207181,ro-crate-ruby,"information, 0.08174\ndata science, 0.00000\ng..."
3,212556220,Misc_Training_scripts,"virtuoso, 0.04775\nvirtuoso, 0.02404\ndocument..."
4,155879756,FAIRifier,"database, 0.13379\ntextile, 0.00004\nHyperText..."
5,90349931,elda,"document, 0.23375\ntextile, 0.00000\ngoods, 0...."
6,126633812,music-genre-classification,"music, 0.02499\nmusic, 0.00000\ncosmology, 0.0..."
7,173520377,probabilistic_nlg,"cosmology, 0.05597\nmusic, 0.00000\ncatalogue,..."
8,103798851,DataStructures-Algorithms-InC,"computer science, 0.06299\ndata science, 0.000..."
9,153249816,Music-Generation-Using-Deep-Learning,"music, 0.08406\ndata science, 0.00000\ngoods, ..."
